In [3]:
import pandas as pd
import inspect
from inspect import signature

from pandas.core.series import Series
from pandas.core.frame import DataFrame

import json

from flask import jsonify

In [4]:
def get_functions(function_list):
    function_classes = function_list
    result_dict = {}
    for function_class in function_classes:
        method_list = [method for method in dir(function_class) if (not method.startswith('__')) and (not method.startswith('OUT'))]
        method_dict = {}

        for method in method_list:
            attr_dict = {}
            method = eval(function_class.__name__ + "." + method)
            sig = signature(method)
            
            parmeter_dict = {}
                              
            for param in sig.parameters.values():
                if not(param.name == "df" or param.name == "output"):
                    parmeter_dict[param.name] = {}
                    
                    if 'empty' in str(param.default):
                        parmeter_dict[param.name]["inputValue"] = None
                    else:
                        parmeter_dict[param.name]["inputValue"] = param.default
                        
                    if 'empty' in str(param.annotation):
                        parmeter_dict[param.name]["inputType"] = None
                    else:
                        parmeter_dict[param.name]["inputType"] = str(param.annotation).split()[1][1:-2]
                    #attr_dict["kind"].append(param.kind)
                    
#                 elif param.name == "output":
#                     attr_dict["outputType"] = param.default
                    
            attr_dict["parameters"] = parmeter_dict 
            #attr_dict["comment"] = inspect.getdoc(method)
            
            comment = inspect.getdoc(method)
            comment = comment.strip().split("\n")
            
            #print(comment)
            
            for s in comment:
                if ":" in s:
                    tmp = s.split(":")
                    attr_dict[tmp[0].strip()] = tmp[1].strip()
                    print(tmp[0].strip())
                    print(tmp[1].strip())
                    
            attr_dict["len"] = method.__code__.co_argcount - 1
            #attr_dict["source"] = inspect.getsource(method)

            method_dict[method.__name__] = attr_dict

            result_dict[function_class.__name__] = method_dict
            
    return result_dict

In [5]:
class UserDefinedRunFunction():
#     OUT_REPLACE = "replace"
#     OUT_CONCAT = "concat"
#     OUT_NONE = "none"

    def one_hot_encoding(df, col:str):
        """
        name : 원 핫 인코딩
        output_type : concat
        comment : 원핫인코딩 함수, 꼭 1개의 인자를 넣어주세요
        """
        return pd.get_dummies(df[col])
    
    def dropna(df, cols:list):
        # df.dropna(subset = ["Sex", "Cabin"], thresh=2400, how='any')
        """
        name : 결측치 제거
        outputType : none
        comment : 결측치 제거 함수, 꼭 1개의 인자를 넣어주세요
        """
        return df.dropna(subset=cols, axis=1)
    
    def scaling(df, how:tuple = ["minmax", "standard"]):
        """
        name : 스케일링
        output_type : none
        comment : 스케일링 함수, 꼭 1개의 인자를 넣어주세요
        """
        if how == "minmax":
            scaler = MinMaxScaler
            return scaler.fit_transform(df)
        elif how == "standard":
            scaler = StandardScaler
            return scaler.fit_transform(df)

In [6]:
class UserDefinedApplyFunction():
    
    def plus(df, col1:int, col2:int):
        """
        name : 컬럼 더하기
        output_type : replace
        comment : 두개의 인자 값을 더해주는 함수! 꼭 2개의 인자를 넣어주세요."""
        return df[col1] + df[col2]

    # columns을 던질때 복수형이면서 int형은 어떻게 처리할 것 인지
    def substr(df, col:int, num:int):
        """
        name : 문자 슬라이싱
        output_type : replace
        comment : 문자를 자르는 함수 꼭 1개의 인자를 넣어주세요.
        """
        return df[col][:-int(num)]

In [7]:
function_list = [UserDefinedRunFunction, UserDefinedApplyFunction]
#function_list = [UserDefinedApplyFunction]

result_json = get_functions(function_list)

name
결측치 제거
output_type
none
comment
결측치 제거 함수, 꼭 1개의 인자를 넣어주세요
name
원 핫 인코딩
output_type
concat
comment
원핫인코딩 함수, 꼭 1개의 인자를 넣어주세요
name
스케일링
output_type
none
comment
스케일링 함수, 꼭 1개의 인자를 넣어주세요
name
컬럼 더하기
output_type
replace
comment
두개의 인자 값을 더해주는 함수! 꼭 2개의 인자를 넣어주세요.
name
문자 슬라이싱
output_type
replace
comment
문자를 자르는 함수 꼭 1개의 인자를 넣어주세요.


In [8]:
result_json

{'UserDefinedRunFunction': {'dropna': {'parameters': {'cols': {'inputValue': None,
     'inputType': 'list'}},
   'name': '결측치 제거',
   'output_type': 'none',
   'comment': '결측치 제거 함수, 꼭 1개의 인자를 넣어주세요',
   'len': 1},
  'one_hot_encoding': {'parameters': {'col': {'inputValue': None,
     'inputType': 'str'}},
   'name': '원 핫 인코딩',
   'output_type': 'concat',
   'comment': '원핫인코딩 함수, 꼭 1개의 인자를 넣어주세요',
   'len': 1},
  'scaling': {'parameters': {'how': {'inputValue': ['minmax', 'standard'],
     'inputType': 'tuple'}},
   'name': '스케일링',
   'output_type': 'none',
   'comment': '스케일링 함수, 꼭 1개의 인자를 넣어주세요',
   'len': 1}},
 'UserDefinedApplyFunction': {'plus': {'parameters': {'col1': {'inputValue': None,
     'inputType': 'int'},
    'col2': {'inputValue': None, 'inputType': 'int'}},
   'name': '컬럼 더하기',
   'output_type': 'replace',
   'comment': '두개의 인자 값을 더해주는 함수! 꼭 2개의 인자를 넣어주세요.',
   'len': 2},
  'substr': {'parameters': {'col': {'inputValue': None, 'inputType': 'int'},
    'num': {'inputVa

In [9]:
with open("./data/user_defined_function.json", "w", encoding="UTF-8") as json_file:
    json.dump(result_json, json_file, ensure_ascii=False)

In [10]:
get_functions(function_list)

name
결측치 제거
output_type
none
comment
결측치 제거 함수, 꼭 1개의 인자를 넣어주세요
name
원 핫 인코딩
output_type
concat
comment
원핫인코딩 함수, 꼭 1개의 인자를 넣어주세요
name
스케일링
output_type
none
comment
스케일링 함수, 꼭 1개의 인자를 넣어주세요
name
컬럼 더하기
output_type
replace
comment
두개의 인자 값을 더해주는 함수! 꼭 2개의 인자를 넣어주세요.
name
문자 슬라이싱
output_type
replace
comment
문자를 자르는 함수 꼭 1개의 인자를 넣어주세요.


{'UserDefinedRunFunction': {'dropna': {'parameters': {'cols': {'inputValue': None,
     'inputType': 'list'}},
   'name': '결측치 제거',
   'output_type': 'none',
   'comment': '결측치 제거 함수, 꼭 1개의 인자를 넣어주세요',
   'len': 1},
  'one_hot_encoding': {'parameters': {'col': {'inputValue': None,
     'inputType': 'str'}},
   'name': '원 핫 인코딩',
   'output_type': 'concat',
   'comment': '원핫인코딩 함수, 꼭 1개의 인자를 넣어주세요',
   'len': 1},
  'scaling': {'parameters': {'how': {'inputValue': ['minmax', 'standard'],
     'inputType': 'tuple'}},
   'name': '스케일링',
   'output_type': 'none',
   'comment': '스케일링 함수, 꼭 1개의 인자를 넣어주세요',
   'len': 1}},
 'UserDefinedApplyFunction': {'plus': {'parameters': {'col1': {'inputValue': None,
     'inputType': 'int'},
    'col2': {'inputValue': None, 'inputType': 'int'}},
   'name': '컬럼 더하기',
   'output_type': 'replace',
   'comment': '두개의 인자 값을 더해주는 함수! 꼭 2개의 인자를 넣어주세요.',
   'len': 2},
  'substr': {'parameters': {'col': {'inputValue': None, 'inputType': 'int'},
    'num': {'inputVa

## 코멘트 파싱 테스트

In [56]:
comment = 'name : 컬럼 더하기\noutput_type : replace\ncomment : 두개의 인자 값을 더해주는 함수! 꼭 2개의 인자를 넣어주세요.'

In [57]:
comment = comment.strip().split("\n")

In [58]:
comment

['name : 컬럼 더하기',
 'output_type : replace',
 'comment : 두개의 인자 값을 더해주는 함수! 꼭 2개의 인자를 넣어주세요.']

In [59]:
for s in comment:
    if ":" in s:
        tmp = s.split(":")
        print(tmp[0].strip())
        print(tmp[1].strip())

name
컬럼 더하기
output_type
replace
comment
두개의 인자 값을 더해주는 함수! 꼭 2개의 인자를 넣어주세요.


## 스케일링 테스트

In [97]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [98]:
df = pd.read_csv("./data/titanic.csv")

In [103]:
def scaling(df, how:tuple = ["minmax", "standard"]):
        """
        name : 스케일링
        output_type : concat
        comment : 스케일링 함수, 꼭 1개의 인자를 넣어주세요
        """
        if how == "minmax":
            scaler = MinMaxScaler
        elif how == "standard":
            scaler = StandardScaler
            
        return scaler.fit_transform(df)

In [104]:
scaling(df, how="minmax")

TypeError: fit_transform() missing 1 required positional argument: 'X'

In [109]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [111]:
df = df.dropna()

In [112]:
scaler = StandardScaler()
scaler.fit_transform(df["Age"])

ValueError: Expected 2D array, got 1D array instead:
array=[38.   35.   54.    4.   58.   34.   28.   19.   49.   65.   45.   29.
 25.   23.   46.   71.   23.   21.   47.   24.   32.5  54.   19.   37.
 24.   36.5  22.   61.   56.   50.    1.    3.   44.   58.    2.   40.
 31.   32.   38.   35.   44.   37.   29.   62.   30.   52.   40.   58.
 35.   37.   63.   19.   36.    2.   50.    0.92 17.   30.   24.   18.
 31.   40.   36.   36.   16.   45.5  38.   29.   41.   45.    2.   24.
 24.   22.   60.   24.   25.   27.   36.   23.   24.   33.   32.   28.
 50.   14.   64.    4.   52.   30.   49.   65.   48.   47.   23.   25.
 35.   58.   55.   54.   25.   16.   18.   36.   47.   34.   30.   44.
 45.   22.   36.   50.   17.   48.   39.   53.   36.   39.   39.   36.
 18.   60.   52.   49.   40.    4.   42.   61.   21.   80.   32.   24.
 48.   56.   58.   47.   31.   36.   27.   15.   31.   49.   42.   18.
 35.   42.   24.   48.   19.   38.   27.   27.   29.   35.   36.   21.
 70.   19.    6.  

In [ ]:
scaler.fi